In [1]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable

from modules import HopfieldPooling

# add path to AttentionDeepMIL-master
import sys
sys.path.append("...path/to/AttentionDeepMIL-master") 
from dataloader import MnistBags
from model import Attention, GatedAttention


AttributeError: module 'torch._jit_internal' has no attribute 'Optional'

In [ ]:
class HfPooling(nn.Module):
    def __init__(self):
        super(HfPooling, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 4 * 4, self.L),
            nn.ReLU(),
        )
        self.hopfield_pooling = HopfieldPooling(input_size=self.L,
                                                hidden_size=32, 
                                                output_size=self.L, 
                                                num_heads=1,
                                                scaling=0.8)
        self.dp = nn.Dropout(p=0.1)
        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.squeeze(0)
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 50 * 4 * 4)
        H = self.feature_extractor_part2(H)  # NxL
        
        H = H.unsqueeze(0)
        H = self.hopfield_pooling(H)
        H = H.squeeze(0)
        H = self.dp(H)

        Y_prob = self.classifier(H)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, None # dummy for A

    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

In [2]:
def train(epoch):
    model.train()
    train_loss = 0.
    train_error = 0.
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        if args.cuda:
            data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)

        # reset gradients
        optimizer.zero_grad()
        # calculate loss and metrics
        loss, _ = model.calculate_objective(data, bag_label)
        train_loss += loss.data[0]
        error, _ = model.calculate_classification_error(data, bag_label)
        train_error += error
        # backward pass
        loss.backward()
        # step
        optimizer.step()

    # calculate loss and error for epoch
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    
    print('Epoch: {}, Loss: {:.4f}, Train error: {:.4f}'.format(epoch, train_loss.cpu().item(), train_error))


def test():
    model.eval()
    test_loss = 0.
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        if args.cuda:
            data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, attention_weights = model.calculate_objective(data, bag_label)
        test_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data, bag_label)
        test_error += error

        #if batch_idx < 5:  # plot bag labels and instance labels for first 5 bags
        #    bag_level = (bag_label.cpu().data.numpy()[0], int(predicted_label.cpu().data.numpy()[0][0]))

    test_error /= len(test_loader)
    test_loss /= len(test_loader)

    print('\nTest Set, Loss: {:.4f}, Test error: {:.4f}'.format(test_loss.cpu().item(), test_error))

In [3]:
class args:
    pass
args.epochs = 20
args.lr = 0.0005
args.reg = 10e-5
args.target_number = 9
args.mean_bag_length = 10
args.var_bag_length = 2
args.num_bags_train = 200
args.num_bags_test = 50
args.seed = 1
args.no_cuda = True
args.cuda = not args.no_cuda
args.model = 'attention'

print('Init Model')
torch.manual_seed(args.seed)

if args.model=='attention':
    model = Attention()
elif args.model=='gated_attention':
    model = GatedAttention()
elif args.model == 'hf_pooling':
    model = HfPooling()
if args.cuda:
    model.cuda()

Init Model


NameError: name 'Attention' is not defined

In [5]:

if args.cuda:
    torch.cuda.manual_seed(args.seed)
    print('\nGPU is ON!')

print('Load Train and Test Set')
loader_kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = data_utils.DataLoader(MnistBags(target_number=args.target_number,
                                               mean_bag_length=args.mean_bag_length,
                                               var_bag_length=args.var_bag_length,
                                               num_bag=args.num_bags_train,
                                               seed=args.seed,
                                               train=True),
                                     batch_size=1,
                                     shuffle=True,
                                     **loader_kwargs)

test_loader = data_utils.DataLoader(MnistBags(target_number=args.target_number,
                                              mean_bag_length=args.mean_bag_length,
                                              var_bag_length=args.var_bag_length,
                                              num_bag=args.num_bags_test,
                                              seed=args.seed,
                                              train=False),
                                    batch_size=1,
                                    shuffle=False,
                                    **loader_kwargs)

optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=args.reg)

Load Train and Test Set


In [6]:
print('Start Training')
for epoch in range(1, args.epochs + 1):
    train(epoch)
print('Start Testing')
test()

Start Training
Epoch: 1, Loss: 0.6899, Train error: 0.4150
Epoch: 2, Loss: 0.6561, Train error: 0.3750
Epoch: 3, Loss: 0.5644, Train error: 0.3250
Epoch: 4, Loss: 0.3412, Train error: 0.1550
Epoch: 5, Loss: 0.2279, Train error: 0.0700
Epoch: 6, Loss: 0.0830, Train error: 0.0300
Epoch: 7, Loss: 0.1755, Train error: 0.0500
Epoch: 8, Loss: 0.0384, Train error: 0.0100
Epoch: 9, Loss: 0.0434, Train error: 0.0250
Epoch: 10, Loss: 0.1016, Train error: 0.0350
Epoch: 11, Loss: 0.0499, Train error: 0.0150
Epoch: 12, Loss: 0.0150, Train error: 0.0050
Epoch: 13, Loss: 0.0053, Train error: 0.0000
Epoch: 14, Loss: 0.0007, Train error: 0.0000
Epoch: 15, Loss: 0.0005, Train error: 0.0000
Epoch: 16, Loss: 0.0003, Train error: 0.0000
Epoch: 17, Loss: 0.0002, Train error: 0.0000
Epoch: 18, Loss: 0.0002, Train error: 0.0000
Epoch: 19, Loss: 0.0002, Train error: 0.0000
Epoch: 20, Loss: 0.0001, Train error: 0.0000
Start Testing

Test Set, Loss: 0.6883, Test error: 0.1000


In [18]:
# RESULTS

# hop.pooling    : Test Set, Loss: 0.2083, Test error: 0.0600
# gated attention: Test Set, Loss: 0.5023, Test error: 0.0800
# attention      : Test Set, Loss: 0.6883, Test error: 0.1000